<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Supp_Fig_4/Supp_Fig_4c/show_primer_bias_splitcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Align SPLIT-Seq data from lung samples from mice infected with SARS-CoV-2
Reference: https://doi.org/10.1038/s41586-022-05344-2

You need Google Colab Pro to run this notebook since the required disk space exceeds the disk space included with the free version.

### Install software

In [1]:
!pip install -q kb_python anndata
import numpy as np
import anndata
import pandas as pd
import json
import os
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.5/36.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.8 MB/s eta 0:00:00


In [2]:
# Download the customized transcripts to gene mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
# Download the RdRP amino acid sequences
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

virus_fasta = "palmdb_rdrp_seqs.fa"
virus_t2g = "palmdb_clustered_t2g.txt"

--2025-02-18 17:09:39--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4561689 (4.3M) [text/plain]
Saving to: ‘palmdb_clustered_t2g.txt’

palmdb_clustered_t2 100%[===================>]   4.35M  --.-KB/s    in 0.06s   

2025-02-18 17:09:39 (75.9 MB/s) - ‘palmdb_clustered_t2g.txt’ saved [4561689/4561689]

--2025-02-18 17:09:40--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaitin

In [3]:
# Number of threads to use in alignment
threads = 2

## Download raw data

In [4]:
!pip install -q ffq
import json

out = "data.json"

# # Download the complete dataset (too large for a standard Colab session)
# !ffq GSE199498 --ftp -o $out

# Download a few example libraries
!ffq SRR18496012 SRR18496013 SRR18496019 --ftp -o $out

f = open(out)
data = json.load(f)
f.close()

print(len(data))

for dataset in data:
    url = dataset["url"]
    !curl -O $url

[2025-02-18 17:09:51,304]    INFO Parsing run SRR18496012
[2025-02-18 17:09:53,769]    INFO Parsing run SRR18496013
[2025-02-18 17:09:56,110]    INFO Parsing run SRR18496019
6
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.1G  100 16.1G    0     0  26.1M      0  0:10:32  0:10:32 --:--:-- 26.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.1G  100 17.1G    0     0  27.1M      0  0:10:47  0:10:47 --:--:-- 28.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9923M  100 9923M    0     0  28.2M      0  0:05:51  0:05:51 --:--:-- 30.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 

## Align data to PalmDB using kallisto translated search

Generate virus index with masked host (here, mouse) genome and transcriptome sequences:

In [5]:
# Get host genome and transcriptome
!pip install -q gget
!gget ref -w cdna,dna -r 110 -d mouse
host_cdna = "Mus_musculus.GRCm39.cdna.all.fa.gz"
host_dna = "Mus_musculus.GRCm39.dna.primary_assembly.fa.gz"

# Concatenate host genome and transcriptome into a single file
host_combined = "combined.cdna_dna.all.fa.gz"
!cat $host_cdna $host_dna > $host_combined

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.2 MB/s eta 0:00:00
17:51:26 - INFO - Fetching reference information for mus_musculus from Ensembl release: 110.
{
    "mus_musculus": {
        "transcriptome_cdna": {
            "ftp": "http://ftp.ensembl.org/pub/release-110/fasta/mus_musculus/cdna/Mus_musculus.GRCm39.cdna.all.fa.gz",
            "ensembl_release": 110,
            "release_date": "2023-04-22",
            "release_time": "03:01",
            "bytes": "49M"
        },
        "genome_dna": {
            "ftp": "http://ftp.ensembl.org/pub/release-110/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz",
            "ensembl_release": 110,
            "release_date": "2023-04-21",
            "release_time": "17:21",
            "bytes": "769M"
        }
    }
}
  % Total    % Received % X

In [ ]:
# Generate virus reference index
virus_index = "virus_index.idx"

!kb ref \
    --aa \
    -t $threads \
    --d-list $host_combined \
    --workflow custom \
    -i $virus_index \
    $virus_fasta

# Optional: delete host genome and transcriptome files after generating the index to save disk space
!rm $host_cdna $host_dna $host_combined

[2025-02-18 17:52:16,486]    INFO [ref_custom] Indexing palmdb_rdrp_seqs.fa to virus_index.idx


Get fastq files:

In [ ]:
import os
import glob

In [ ]:
fastqs = []
for filename in glob.glob("*.fastq.gz"):
    fastqs.append(filename.split("/")[-1])

fastqs.sort()
fastqs

In [ ]:
len(fastqs)

In [ ]:
samples = []
for fastq in fastqs:
    samples.append(fastq.split("_")[0])

In [ ]:
samples = list(set(samples))
len(samples)

Align data to PalmDB:  
The SPLIT-Seq barcode onlist files (r1_RT_replace.txt and r1r2r3.txt) were provided by Delaney Sullivan (07/15/2023).

In [ ]:
# Download SPLIT-Seq barcode onlist files
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Supp_Fig_4/Supp_Fig_4c/r1_RT_replace.txt
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Supp_Fig_4/Supp_Fig_4c/r1r2r3.txt

In [ ]:
%%time
out_folder = "palmdb"
for sample in samples:
    fastq1 = sample + "_1.fastq.gz"
    fastq2 = sample + "_2.fastq.gz"

    !mkdir -p $out_folder/$sample

    !kb count \
        --aa \
        --h5ad \
        -t $threads \
        -i $virus_index \
        -g $virus_t2g \
        -x SPLIT-Seq \
        -r r1_RT_replace.txt \
        -w r1r2r3.txt \
        -o $out_folder/$sample/ \
        $fastq1 $fastq2

    # Optional: Delete raw sequencing data after alignment to save disk space
    !rm $fastq1 $fastq2

## Add metadata

In [ ]:
# Get SRRs
srrs = []
for filename in glob.glob("*.fastq.gz"):
    srrs.append(filename.split("/")[-1].split("_")[0])

srrs = list(set(srrs))

srrs.sort()
print(len(srrs))
srrs

Add primer type and well from onlist based on the last 8 bases of barcode:

In [ ]:
# Download barcode to well mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Supp_Fig_4/Supp_Fig_4c/splitseq_bc2well.csv
bc2well = pd.read_csv("splitseq_bc2well.csv")
bc2well

In [ ]:
palmdb_adatas = []
for srr in srrs:
    # Load data
    adata = anndata.read(f"{out_folder}/{srr}/counts_unfiltered/adata.h5ad")

    # Add sample name
    adata.obs["srr"] = srr

    # Create new column containing identifying barcode
    adata.obs["iden_bc"] = adata.obs.index.str[-8:]

    # Create copy of barcode column
    adata.obs["barcode"] = adata.obs.index

    # Merge well and primer information
    adata.obs = adata.obs.merge(bc2well, left_on="iden_bc", right_on="sequence", how="left").set_index("barcode", drop=False)

    # Append to adata list
    palmdb_adatas.append(adata)

In [ ]:
## Concatenate datasets
palmdb_adata = palmdb_adatas[0]
for adata_obj in palmdb_adatas[1:]:
    palmdb_adata = palmdb_adata.concatenate(adata_obj,
                             join='outer',
                             batch_key='sample_index',
                             batch_categories=None,
                             uns_merge=None,
                             fill_value=np.nan)

palmdb_adata

In [ ]:
# Drop unnecessary/duplicated columns
palmdb_adata.obs = palmdb_adata.obs.drop(["sequence", "sample_index"], axis=1)
palmdb_adata.obs

Add metadata:

In [ ]:
# Load first set of metadata provided by authors
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Supp_Fig_4/Supp_Fig_4c/GSM5974202_sample_well_IDs.tsv
meta1 = pd.read_csv("GSM5974202_sample_well_IDs.tsv", sep='\t')
meta1["well_ID_list"] = meta1["well_ID"]
meta1["well_ID"] = meta1["well_ID_list"].str.split("-")
meta1 = meta1.explode("well_ID")
meta1.head()

In [ ]:
# Copy current index
palmdb_adata.obs["barcode_idx"] = palmdb_adata.obs.index.values

In [ ]:
palmdb_adata.obs = palmdb_adata.obs.merge(meta1, left_on="well", right_on="well_ID", how="left").drop("well_ID_list", axis=1)
palmdb_adata.obs

In [ ]:
# Load second set of metadata provided by authors
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Supp_Fig_4/Supp_Fig_4c/GSM5974202_README.txt
meta2 = pd.read_csv("GSM5974202_README.txt", sep='\t')
meta2.head()

In [ ]:
# Merge second set of metadata into adata object
palmdb_adata.obs = palmdb_adata.obs.merge(meta2, left_on="sample", right_on="Sample_name", how="left").set_index("barcode_idx", drop=True)
palmdb_adata.obs

## Plot fractions of virus counts detected per primer type

In [ ]:
# Download virus ID to sOTU mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
id2tax_df = pd.read_csv("ID_to_taxonomy_mapping.csv")
id2tax_df.head()

In [ ]:
# Minimum count
threshold_per_cell = 0

In [ ]:
# Get fraction of positive cells per virus strandedness per primer type
vtypes = ['+ssRNA', '-ssRNA', 'dsRNA', 'dsDNA', '+ssRNA or dsRNA']
vlabels = ['+ssRNA', '-ssRNA', 'dsRNA', 'dsDNA', '+ssRNA\nor dsRNA']

t_counts = []
r_counts = []
for vt in vtypes:
    target_ids = id2tax_df[id2tax_df["virus_type"] == vt]["Label"].values

    # Get counts for poly T primer
    tc = nd(palmdb_adata[(palmdb_adata.obs["type"]=="T") & (palmdb_adata.obs["condition"] == "infected"), palmdb_adata.var.index.isin(target_ids)].X.todense())
    tc = tc[tc > threshold_per_cell]

    # Get counts for random primers
    rc = nd(palmdb_adata[(palmdb_adata.obs["type"]=="R") & (palmdb_adata.obs["condition"] == "infected"), palmdb_adata.var.index.isin(target_ids)].X.todense())
    rc = rc[rc > threshold_per_cell]

    t_counts.append(np.sum(tc) / (np.sum(tc) + np.sum(rc)))
    r_counts.append(np.sum(rc) / (np.sum(tc) + np.sum(rc)))

In [ ]:
# Get fraction of positive cells for SARS-CoV-2 per primer type
species = [
    "Severe acute respiratory syndrome-related coronavirus",
]

labels = [
    "Severe acute respiratory\nsyndrome-related\ncoronavirus",
]

t_counts_sars = []
r_counts_sars = []
total_count = []
for sp in species:
    target_ids = id2tax_df[id2tax_df["species"] == sp]["rep_ID"].values

    # Get counts for poly T primer
    tc = nd(palmdb_adata[(palmdb_adata.obs["type"]=="T") & (palmdb_adata.obs["condition"] == "infected"), palmdb_adata.var.index.isin(target_ids)].X.todense())
    tc = tc[tc > threshold_per_cell]

    # Get counts for random primers
    rc = nd(palmdb_adata[(palmdb_adata.obs["type"]=="R") & (palmdb_adata.obs["condition"] == "infected"), palmdb_adata.var.index.isin(target_ids)].X.todense())
    rc = rc[rc > threshold_per_cell]

    t_counts_sars.append(np.sum(tc) / (np.sum(tc) + np.sum(rc)))
    r_counts_sars.append(np.sum(rc) / (np.sum(tc) + np.sum(rc)))

    total_count.append(np.sum(tc) + np.sum(rc))

In [ ]:
fig, axs = plt.subplots(figsize=(7, 7), ncols=2, sharey=True, gridspec_kw={'width_ratios': [1, 5], 'wspace': 0.05})

fontsize = 14
width = 0.4
colors = ["#003049", "#98c1d9"]

# Plot SARS-CoV2
ax = axs[0]
ax.bar(0-width/2, r_counts_sars[0], width=width, color=colors[0])
ax.bar(0+width/2, t_counts_sars[0], width=width, color=colors[1])
ax.set_xticks([0], ["SARS-CoV"], fontsize=fontsize)

# ax.set_xlabel("Known viral infection", fontsize=fontsize)
ax.set_ylabel("Fraction of kallisto counts", fontsize=fontsize)
ax.set_xmargin(0.1)

# Plot general strandedness
ax = axs[1]
x = np.arange(len(vtypes))
ax.bar(x-width/2, r_counts, width=width, color=colors[0], label="Random\nhexamer")
ax.bar(x+width/2, t_counts, width=width, color=colors[1], label="Poly(T)")

ax.legend(title="Primer type", fontsize=fontsize, title_fontsize=fontsize)
ax.set_xticks(np.arange(len(vlabels)), vlabels, fontsize=fontsize)
ax.set_xmargin(0.02)

for ax in axs:
    ax.tick_params(axis='y', labelsize=fontsize)

fig.suptitle("Lung samples from mice infected\nwith SARS-CoV-2 (SPLiT-seq)", fontsize=fontsize, y=0.95)

fig.savefig("random_vs_polyT_final.png", dpi=300, bbox_inches="tight")

fig.show()